# Práctico 1

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

## Importaciones

In [17]:
import pandas as pd
import numpy as np

## Constantes

In [2]:
SET_DE_ENTRENAMIENTO = './data/meli-challenge-2019/spanish.train.jsonl.gz'
SET_DE_PRUEBA = './data/meli-challenge-2019/spanish.test.jsonl.gz'
SET_DE_VALIDACION = './data/meli-challenge-2019/spanish.validation.jsonl.gz'

## Carga de datos

In [29]:
file_paths = [SET_DE_ENTRENAMIENTO, # Ingresar opción 0 👁 ⚠ Puede que haya un desbordamiento de RAM ⚠
              SET_DE_PRUEBA,  # Ingresar opción 1
              SET_DE_VALIDACION, # Ingresar opción 2
              './data/meli-challenge-2019/spanish_token_to_index.json.gz'] # Ingresar opción 3
i = int(input('Ingresar opción para carga de archivo (0 a 3): '))
df = pd.read_json(path_or_buf=file_paths[i], lines=True)

Ingresar opción para carga de archivo (0 a 3): 3


## Análisis y visualización de los datos

El **set de entrenamiento** original tiene 4895280 registros y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 4508043 registros no confiables y 387237 registros confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _train_ para el set de entrenamiento.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado. Esta información es la que se utilizará para entrenar el modelo.
- **target**: El número que corresponde a cada categoría.
- n_labels
- **size**: La cantidad de registros. 4895280 registros para el caso del set de entrenamiento.

El **set de prueba** original tiene 63680 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Todas las etiquetas de este dataset son confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _test_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado. Esta información es la que se utilizará para entrenar el modelo.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéticas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de prueba.
- **size**: La cantidad de registros. 63680 registros para el caso del set de prueba.

El **set de validación** original tiene 1223820 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 1127189 registros no confiables y 96631 registros confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _validation_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado. Esta información es la que se utilizará para entrenar el modelo.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéticas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de prueba.
- **size**: La cantidad de registros. 1223820 registros para el caso del set de validación.

El archivo **spanish_token_to_index** tiene las 50002 correspondencias que existen entre las palabras tokenizadas del título y las etiquetas numéricas bajo la colimna data en los sets de entrenamiento, prueba y validación.

In [30]:
df.head()

,barbies,casita,muñecas,pintadas,cromado,holográfico,neceser,asiento,chevrolet,funda,...,chorros,budadá,cáp,sebium,hlertfx,promix,acrilab,galapagos,[PAD],[UNK]
0,50000,50001,2,3,4,5,6,7,8,9,...,49992,49993,49994,49995,49996,49997,49998,49999,0,1


In [24]:
df.columns

Index(['language', 'label_quality', 'title', 'category', 'split',
       'tokenized_title', 'data', 'target', 'n_labels', 'size'],
      dtype='object')

In [31]:
df.shape

(1, 50002)

In [25]:
# df.language.value_counts()
df.label_quality.value_counts()

unreliable    1127189
reliable        96631
Name: label_quality, dtype: int64

In [26]:
df.isnull().sum()

language           0
label_quality      0
title              0
category           0
split              0
tokenized_title    0
data               0
target             0
n_labels           0
size               0
dtype: int64

In [27]:
labels = df.target.unique()
print(f'{labels.shape[0]} etiquetas ordenadas como se muestra a continuación:')
np.sort(labels)

632 etiquetas ordenadas como se muestra a continuación:


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18